In [1]:
import sys
import pandas as pd
import numpy as np

In [2]:
## Reading processed datasets
df_cv_goodbad= pd.read_pickle("df_goodbad_filtered.pkl")
df_cv_retro = pd.read_pickle("df_retro_filtered.pkl")

df_cv_goodbad.shape, df_cv_retro.shape

((83364, 48), (869346, 42))

In [3]:

# Restricting Scrub loans to those of driver set LOS APP IDs

firstdate_curr_scrub = df_cv_goodbad.groupby('LOS_APP_ID').agg({'DISBURSED_DT' : min}).reset_index()
firstdate_curr_scrub.rename(columns = {'DISBURSED_DT':'DISBURSED_DT_curr_scrub'}, inplace = True)

df_cv_retro= df_cv_retro.merge(firstdate_curr_scrub, how="inner" , on ="LOS_APP_ID")


from datetime import date
df_cv_retro['DISBURSED_DT'] = pd.to_datetime(df_cv_retro['DISBURSED_DT'],format="%d-%m-%Y",errors='coerce')
df_cv_retro['DATE_REPORTED'] = pd.to_datetime(df_cv_retro['DATE_REPORTED'],format="%d-%m-%Y",errors='coerce')
df_cv_retro['DISBURSED_DT_curr_scrub'] = pd.to_datetime(df_cv_retro['DISBURSED_DT_curr_scrub'],format="%d-%m-%Y",errors='coerce')
df_cv_retro_useful = df_cv_retro[df_cv_retro['DISBURSED_DT'] < df_cv_retro['DISBURSED_DT_curr_scrub'] ]

df_cv_retro.shape,df_cv_retro_useful.shape



((869346, 43), (734211, 43))

In [4]:

!pip install shutup
import shutup 
shutup.please()


In [5]:
df_cv_retro_useful.columns

Index(['CREDT_RPT_ID', 'LOS_APP_ID', 'CANDIDATE__ID', 'CUSTOMER_ID_MBR_ID',
       'BRANCH', 'KENDRA', 'SELF_INDICATOR', 'MATCH_TYPE', 'ACC_NUM',
       'CREDIT_GRANTOR', 'ACCT_TYPE', 'CONTRIBUTOR_TYPE', 'DATE_REPORTED',
       'OWNERSHIP_IND', 'ACCOUNT_STATUS', 'DISBURSED_DT', 'CLOSE_DT',
       'LAST_PAYMENT_DATE', 'CREDIT_LIMIT_SANC_AMT',
       '_DISBURSED_AMT_HIGH_CREDIT', '_INSTALLMENT_AMT', '_CURRENT_BAL',
       'INSTALLMENT_FREQUENCY', 'WRITE_OFF_DATE', '_OVERDUE_AMT',
       '_WRITE_OFF_AMT', 'ASSET_CLASS', '_ACCOUNT_REMARKS', 'LINKED_ACCOUNTS',
       'REPORTED_DATE__HIST_', 'DPD__HIST', 'ASSET_CLASS__HIST_',
       'HIGH_CRD__HIST_', 'CUR_BAL__HIST_', 'DAS__HIST_', 'AMT_OVERDUE__HIST_',
       'AMT_PAID__HIST_', 'INCOME_', '_INCOME_INDICATOR_', 'TENURE_',
       '_OCCUPATION', 'Unnamed_41', 'DISBURSED_DT_curr_scrub'],
      dtype='object')

In [9]:
df_cv_retro_useful.ACCT_TYPE.unique()

array(['Commercial Vehicle Loan', 'Other', 'Auto Loan (Personal)',
       'Overdraft', 'Education Loan', 'Loan Against Bank Deposits',
       'Business Loan Priority Sector  Agriculture', 'Gold Loan',
       'Housing Loan', 'Loan Against Shares / Securities',
       'Business Loan General', 'Property Loan', 'Consumer Loan',
       'Two-Wheeler Loan', 'Credit Card', 'Used Car Loan', 'Fleet Card',
       'Tractor Loan', 'Personal Loan', 'Construction Equipment Loan',
       'Business Loan Priority Sector  Others', 'Kisan Credit Card',
       'Business Loan Priority Sector  Small Business',
       'Mudra Loans - Shishu / Kishor / Tarun', 'Loan to Professional',
       'Non-Funded Credit Facility', 'Commercial Equipment Loan',
       'Business Non-Funded Credit Facility-Priority Sector- Small Business',
       'Business Non-Funded Credit Facility General',
       'Business Loan - Secured', 'Individual', 'JLG Individual',
       'Business Loan Against Bank Deposits', 'JLG Group',
       'Pr

In [14]:

df_cv_retro_useful[df_cv_retro_useful['ACCT_TYPE'] == 'Commercial Vehicle Loan'].shape ,df_cv_retro_useful[df_cv_retro_useful['ACCT_TYPE'] == 'Commercial Vehicle Loan'].LOS_APP_ID.nunique()


((323928, 43), 28678)

In [155]:
# Breaking down datasets based on number of historical loans

df_cv_retro_useful['Loan_id'] = df_cv_retro_useful.reset_index().index
customer_loans=df_cv_retro_useful.groupby('LOS_APP_ID')["Loan_id"].count().reset_index()
#c1  = customer_loans[(customer_loans['Loan_id']<20)]
#c1  = customer_loans[(customer_loans['Loan_id']>=20)& (customer_loans['Loan_id']<40)]
#c1  = customer_loans[(customer_loans['Loan_id']>=40)& (customer_loans['Loan_id']<80)]
c1  = customer_loans[(customer_loans['Loan_id']>=80)]
c1.shape, customer_loans.shape

((1519, 2), (33679, 2))

In [125]:
# Filtering Retro scrub based on identified shards
retro1  = df_cv_retro_useful.merge(c1[['LOS_APP_ID']], on = "LOS_APP_ID" , how= "inner")
retro1.shape, df_cv_retro_useful.shape

((181237, 46), (734211, 46))

# Features based on DPD, Current Balance and Overdue Amount Strings

In [126]:

# Breaking Down DPD, urrent balance and Overdue Amount strings

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

cols = ['Loan_id',"DISBURSED_DT", "CONTRIBUTOR_TYPE", "ACCT_TYPE", "DATE_REPORTED" ,"DISBURSED_DT_curr_scrub",
        '_DISBURSED_AMT_HIGH_CREDIT','LOS_APP_ID' , "ACCOUNT_STATUS", "REPORTED_DATE__HIST_","DPD__HIST",
       "AMT_OVERDUE__HIST_","CUR_BAL__HIST_"]

temp = retro1[cols]
#temp['DPD__HIST'] = temp['DPD__HIST'].astype(str) 
for n in range(37):
    #print(n+1)
    st= n*3
    lst =(n+1)*3
    #print(st,lst)
    temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
    #temp['DPD' + str(n+1)]= pd.to_numeric( temp['DPD' + str(n+1)], errors = "coerce")
    st= n*8 + n
    lst =(n+1)*8 + n
    #print(st,lst)
    temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
    temp['Reported_Dt' + str(n+1)] = pd.to_datetime(temp['Reported_Dt' + str(n+1)],format="%Y%m%d",errors='coerce')


met = 'AMT_OVERDUE'
col ="AMT_OVERDUE__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)


met = 'CUR_BAL'
col ="CUR_BAL__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)

print(temp[['CUR_BAL__HIST_','CUR_BAL1','CUR_BAL2','CUR_BAL3']].head(4))

print(temp[['REPORTED_DATE__HIST_', 'Reported_Dt1','Reported_Dt2']].head(2))

(181237, 87)
(181237, 124)
                                      CUR_BAL__HIST_ CUR_BAL1 CUR_BAL2  \
0               0,32221,358221,396181,415161,434141,        0    32221   
1  0,137284,908284,942784,977264,1011764,1046264,...        0   137284   
2  0,100180,668630,694130,719610,745110,770610,79...        0   100180   
3                       0,0,23540,47120,70720,94320,        0        0   

  CUR_BAL3  
0   358221  
1   908284  
2   668630  
3    23540  
                                REPORTED_DATE__HIST_ Reported_Dt1 Reported_Dt2
0  20141231,20141130,20141031,20140930,20140831,2...   2014-12-31   2014-11-30
1  20150228,20150131,20141231,20141130,20141031,2...   2015-02-28   2015-01-31


In [127]:
# Transpose Wide to Long


temp.drop(['AMT_OVERDUE__HIST_', 'CUR_BAL__HIST_','REPORTED_DATE__HIST_',"DATE_REPORTED", 'ACCOUNT_STATUS',"DPD__HIST"], axis=1,inplace = True )
#new_df = temp.head(100)

long= pd.wide_to_long(  temp, 
                        stubnames=['Reported_Dt','DPD','CUR_BAL','AMT_OVERDUE'],
                        i=['Loan_id','LOS_APP_ID' ,"DISBURSED_DT", "CONTRIBUTOR_TYPE", 
                           "ACCT_TYPE" ,"DISBURSED_DT_curr_scrub"], 
                        j='Counter').reset_index()


print(long.shape)
long.dropna(subset=['Reported_Dt'], inplace=True)
print(long.shape)


(6705769, 12)
(2492010, 12)


In [129]:

# Creating granular features at a Bank , Acc Type level
metrics = ['DPD','CUR_BAL','AMT_OVERDUE']
long['Mont_diff'] = ((pd.to_datetime(long.DISBURSED_DT_curr_scrub) - 
                                    pd.to_datetime(long.Reported_Dt) )/np.timedelta64(1, 'M'))

periods = [0,6,12,18,24,30]
for met in metrics:
    for pos in range( len(periods) - 1 ):
        startm = periods[pos]
        endm = periods[pos+1]

        #temp_l['AMT_OVERDUE__HIST_'] = temp_l['AMT_OVERDUE__HIST_'].str.replace("",'0')
        long[met] = pd.to_numeric(long[met], errors= "coerce")

        feat_temp = long[(long['Mont_diff'] > startm) & (long['Mont_diff'] <= endm)].groupby(['LOS_APP_ID','Loan_id','ACCT_TYPE','CONTRIBUTOR_TYPE']).agg(
            {
                met : max,
            }).reset_index()

        feat = feat_temp.groupby(['LOS_APP_ID','ACCT_TYPE','CONTRIBUTOR_TYPE']).agg(
            {
                met  : sum,
            }).reset_index()



        feat.columns = ['LOS_APP_ID',met + '_' + str(startm) + "_" + str(endm) +  'ACCT_TYPE',
                        met + '_' + str(startm) + "_" + str(endm) + 'CONTRIBUTOR_TYPE' , 
                        met + '_' + str(startm) + "_" + str(endm)]
        colnames = [met + '_' + str(startm) + "_" + str(endm) +  'ACCT_TYPE',met + '_' + str(startm) + "_" + str(endm) + 'CONTRIBUTOR_TYPE']

        for col in colnames:       
            feat[col] = feat[col].str.replace(" ", "")
            feat[col] = feat[col].str.replace("-", "_")
            feat[col] = feat[col].str.replace("/", "_")
            feat[col] = feat[col].str.replace(":", "_")
            feat[col] = feat[col].str.replace("(", "")
            feat[col] = feat[col].str.replace(")", "")
            feat[col] = feat[col].str.replace("__", "_")
            feat[col] = feat[col].str.replace("__", "_")

        #print(feat.head(3))


        feat['pivot_col']= feat[colnames[0]] + "_" + feat[colnames[1]]  + "_" + met + "_" +  str(startm) + "_" + str(endm)
        feat = pd.pivot(feat , index=['LOS_APP_ID'], columns= "pivot_col", values=  met + '_' + str(startm) + "_" + str(endm)).reset_index()
        #print(feat.columns)
        #feat.drop(["pivot_col"], axis=1,inplace = True )
        #print(feat.head(2))
        if pos == 0 and met ==metrics[0]:
            feat_vf = feat
        else:
            feat_vf = feat_vf.merge(feat, on ="LOS_APP_ID",how='outer')


        print(feat_vf.shape, feat.shape)


(1190, 87) (1190, 87)
(1301, 175) (1300, 89)
(1379, 259) (1379, 85)
(1419, 342) (1419, 84)
(1459, 419) (1458, 78)
(1459, 505) (1190, 87)
(1459, 593) (1300, 89)
(1459, 677) (1379, 85)
(1459, 760) (1419, 84)
(1459, 837) (1458, 78)
(1459, 923) (1190, 87)
(1459, 1011) (1300, 89)
(1459, 1095) (1379, 85)
(1459, 1178) (1419, 84)
(1459, 1255) (1458, 78)


In [130]:
#feat_vf.to_pickle("features_c_loans_acc_bank_lessthan20.pkl")
#feat_vf.to_pickle("features_c_loans_acc_bank_20_40.pkl")
#feat_vf.to_pickle("features_c_loans_acc_bank_40_80.pkl")
feat_vf.to_pickle("features_c_loans_acc_bank_80_plus.pkl")

feat_vf.head(3)


pivot_col,LOS_APP_ID,AutoLoanPersonal_COP_DPD_0_6,AutoLoanPersonal_NAB_DPD_0_6,AutoLoanPersonal_NBF_DPD_0_6,AutoLoanPersonal_PRB_DPD_0_6,BusinessLoanGeneral_FRB_DPD_0_6,BusinessLoanGeneral_NAB_DPD_0_6,BusinessLoanGeneral_NBF_DPD_0_6,BusinessLoanGeneral_PRB_DPD_0_6,BusinessLoanPrioritySectorAgriculture_NAB_DPD_0_6,...,PropertyLoan_NBF_AMT_OVERDUE_24_30,PropertyLoan_PRB_AMT_OVERDUE_24_30,TractorLoan_NBF_AMT_OVERDUE_24_30,TractorLoan_PRB_AMT_OVERDUE_24_30,Two_WheelerLoan_NAB_AMT_OVERDUE_24_30,Two_WheelerLoan_NBF_AMT_OVERDUE_24_30,Two_WheelerLoan_PRB_AMT_OVERDUE_24_30,Two_WheelerLoan_RRB_AMT_OVERDUE_24_30,UsedCarLoan_NBF_AMT_OVERDUE_24_30,UsedCarLoan_PRB_AMT_OVERDUE_24_30
0,0CV930028R1601088003,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0CV935249F1501029504,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1267925,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# Creating  features Customer Level

metrics = ['DPD','CUR_BAL','AMT_OVERDUE']
long['Mont_diff'] = ((pd.to_datetime(long.DISBURSED_DT_curr_scrub) - 
                                    pd.to_datetime(long.Reported_Dt) )/np.timedelta64(1, 'M'))

periods = [0,6,12,18,24,30]
for met in metrics:
    for pos in range( len(periods) - 1 ):
        startm = periods[pos]
        endm = periods[pos+1]

        #temp_l['AMT_OVERDUE__HIST_'] = temp_l['AMT_OVERDUE__HIST_'].str.replace("",'0')
        long[met] = pd.to_numeric(long[met], errors= "coerce")

        feat_temp = long[(long['Mont_diff'] > startm) & (long['Mont_diff'] <= endm)].groupby(['LOS_APP_ID','Loan_id']).agg(
            {
                met : max,
            }).reset_index()

        feat = feat_temp.groupby(['LOS_APP_ID']).agg(
            {
                met  : sum,
            }).reset_index()



        feat.columns = ['LOS_APP_ID',
                        met + '_' + str(startm) + "_" + str(endm)]
       


        #print(feat.columns)
        #feat.drop(["pivot_col"], axis=1,inplace = True )
        #print(feat.head(2))
        if pos == 0 and met ==metrics[0]:
            feat_vf = feat
        else:
            feat_vf = feat_vf.merge(feat, on ="LOS_APP_ID",how='outer')


        print(feat_vf.shape, feat.shape)


(1190, 2) (1190, 2)
(1301, 3) (1300, 2)
(1379, 4) (1379, 2)
(1419, 5) (1419, 2)
(1459, 6) (1458, 2)
(1459, 7) (1190, 2)
(1459, 8) (1300, 2)
(1459, 9) (1379, 2)
(1459, 10) (1419, 2)
(1459, 11) (1458, 2)
(1459, 12) (1190, 2)
(1459, 13) (1300, 2)
(1459, 14) (1379, 2)
(1459, 15) (1419, 2)
(1459, 16) (1458, 2)


In [132]:
#feat_vf.to_pickle("features_c_loans_lessthan20.pkl")
#feat_vf.to_pickle("features_c_loans_20_40.pkl")
#feat_vf.to_pickle("features_c_loans_40_80.pkl")
feat_vf.to_pickle("features_c_loans_80_plus.pkl")

feat_vf.head()

,LOS_APP_ID,DPD_0_6,DPD_6_12,DPD_12_18,DPD_18_24,DPD_24_30,CUR_BAL_0_6,CUR_BAL_6_12,CUR_BAL_12_18,CUR_BAL_18_24,CUR_BAL_24_30,AMT_OVERDUE_0_6,AMT_OVERDUE_6_12,AMT_OVERDUE_12_18,AMT_OVERDUE_18_24,AMT_OVERDUE_24_30
0,0CV930028R1601088003,171.0,78.0,78.0,49.0,49.0,485515909.0,385367990.0,67287876.0,82078558.0,111207367.0,1200.0,1200.0,1200.0,1144755.0,800.0
1,0CV935249F1501029504,818.0,790.0,630.0,23.0,0.0,121739278.0,130298654.0,57952975.0,3507015.0,8063016.0,15390.0,15012.0,51883.0,39193.0,0.0
2,1267925,30.0,30.0,0.0,0.0,0.0,79621249.0,66803283.0,958319.0,1756584.0,2566422.0,0.0,0.0,0.0,0.0,0.0
3,1287309,30.0,30.0,0.0,0.0,0.0,79621249.0,66803283.0,958319.0,1756584.0,2566422.0,0.0,0.0,0.0,0.0,0.0
4,1287327,30.0,30.0,0.0,0.0,0.0,79621249.0,66803283.0,958319.0,1756584.0,2566422.0,0.0,0.0,0.0,0.0,0.0


In [133]:

#Compiling granular and customer features

#granular_feat  =pd.read_pickle("features_c_loans_acc_bank_lessthan20.pkl")
#compiled_feat = pd.read_pickle("features_c_loans_lessthan20.pkl")
#net_feat = granular_feat.merge(compiled_feat, on ="LOS_APP_ID",how= "outer" )
#net_feat.to_pickle("features_c_loans_all_lessthan20.pkl")
#net_feat.shape,net_feat.LOS_APP_ID.nunique(), granular_feat.shape, compiled_feat.shape


#granular_feat  =pd.read_pickle("features_c_loans_acc_bank_20_40.pkl")
#compiled_feat = pd.read_pickle("features_c_loans_20_40.pkl")
#net_feat = granular_feat.merge(compiled_feat, on ="LOS_APP_ID",how= "outer" )
#net_feat.to_pickle("features_c_loans_all_20_40.pkl")
#net_feat.shape,net_feat.LOS_APP_ID.nunique(), granular_feat.shape, compiled_feat.shape


#granular_feat  =pd.read_pickle("features_c_loans_acc_bank_40_80.pkl")
#compiled_feat = pd.read_pickle("features_c_loans_40_80.pkl")
#net_feat = granular_feat.merge(compiled_feat, on ="LOS_APP_ID",how= "outer" )
#net_feat.to_pickle("features_c_loans_all_40_80.pkl")
#net_feat.shape,net_feat.LOS_APP_ID.nunique(), granular_feat.shape, compiled_feat.shape


granular_feat  =pd.read_pickle("features_c_loans_acc_bank_80_plus.pkl")
compiled_feat = pd.read_pickle("features_c_loans_80_plus.pkl")
net_feat = granular_feat.merge(compiled_feat, on ="LOS_APP_ID",how= "outer" )
net_feat.to_pickle("features_c_loans_all_80_plus.pkl")
net_feat.shape,net_feat.LOS_APP_ID.nunique(), granular_feat.shape, compiled_feat.shape


((1459, 1270), 1459, (1459, 1255), (1459, 16))

# Feature based on Disbursed Amount history

In [138]:
# Creating  features at Customer level

metrics = ['DisbAmt']
df_cv_retro_useful['DisbAmt']= df_cv_retro_useful['_DISBURSED_AMT_HIGH_CREDIT']
df_cv_retro_useful['Mont_since_disb'] = ((pd.to_datetime(df_cv_retro_useful.DISBURSED_DT_curr_scrub) - pd.to_datetime(df_cv_retro_useful.DISBURSED_DT) )/np.timedelta64(1, 'M'))

periods = [0,6,12,18,24,30]
for met in metrics:
    for pos in range( len(periods) - 1 ):
        startm = periods[pos]
        endm = periods[pos+1]

        #temp_l['AMT_OVERDUE__HIST_'] = temp_l['AMT_OVERDUE__HIST_'].str.replace("",'0')
        df_cv_retro_useful[met] = pd.to_numeric(df_cv_retro_useful[met], errors= "coerce")
        

        
        feat = df_cv_retro_useful[(df_cv_retro_useful['Mont_since_disb'] > startm) &
                                       (df_cv_retro_useful['Mont_since_disb'] <= endm)].groupby(['LOS_APP_ID']).agg(
            {
                 met  :  'sum'
               
            }).reset_index()
        
        feat.columns=['LOS_APP_ID',met + '_' + str(startm) + "_" + str(endm)+ "_sum"]
        
        
        #print(feat.columns)
        #feat.drop(["pivot_col"], axis=1,inplace = True )
        #print(feat.head(2))
        if pos == 0 and met ==metrics[0]:
            feat_vf = feat
        else:
            feat_vf = feat_vf.merge(feat, on ="LOS_APP_ID",how='outer')


        print(feat_vf.shape, feat.shape)



(16051, 2) (16051, 2)
(21991, 3) (17435, 2)
(25356, 4) (17888, 2)
(27483, 5) (18004, 2)
(29150, 6) (18203, 2)


In [139]:
feat_vf.to_pickle("features_disbursed_loans.pkl")
feat_vf.head(3)

,LOS_APP_ID,DisbAmt_0_6_sum,DisbAmt_6_12_sum,DisbAmt_12_18_sum,DisbAmt_18_24_sum,DisbAmt_24_30_sum
0,0CV792020R1500937089,0.0,0.0,NaN,NaN,NaN
1,0CV792024R1500968341,0.0,0.0,0.0,0.0,NaN
2,0CV792070R1500935321,335.0,NaN,0.0,0.0,NaN


In [141]:

# Creating granular features at a Bank , Acc Type level

metrics = ['DisbAmt']
df_cv_retro_useful['DisbAmt']= df_cv_retro_useful['_DISBURSED_AMT_HIGH_CREDIT']
df_cv_retro_useful['Mont_since_disb'] = ((pd.to_datetime(df_cv_retro_useful.DISBURSED_DT_curr_scrub) - pd.to_datetime(df_cv_retro_useful.DISBURSED_DT) )/np.timedelta64(1, 'M'))

periods = [0,6,12,18,24,30]
for met in metrics:
    for pos in range( len(periods) - 1 ):
        startm = periods[pos]
        endm = periods[pos+1]

        #temp_l['AMT_OVERDUE__HIST_'] = temp_l['AMT_OVERDUE__HIST_'].str.replace("",'0')
        df_cv_retro_useful[met] = pd.to_numeric(df_cv_retro_useful[met], errors= "coerce")

        
        feat = df_cv_retro_useful[(df_cv_retro_useful['Mont_since_disb'] > startm) &
                                       (df_cv_retro_useful['Mont_since_disb'] <= endm)].groupby(['LOS_APP_ID','ACCT_TYPE','CONTRIBUTOR_TYPE']).agg(
            {
                 met  :  'sum'
               
            }).reset_index()
        
        feat.columns=['LOS_APP_ID','ACCT_TYPE','CONTRIBUTOR_TYPE',
                        met + '_' + str(startm) + "_" + str(endm)+ "_sum"
                       ]
        
        
        colnames = ['ACCT_TYPE','CONTRIBUTOR_TYPE']

        for col in colnames:       
            feat[col] = feat[col].str.replace(" ", "")
            feat[col] = feat[col].str.replace("-", "_")
            feat[col] = feat[col].str.replace("/", "_")
            feat[col] = feat[col].str.replace(":", "_")
            feat[col] = feat[col].str.replace("(", "")
            feat[col] = feat[col].str.replace(")", "")
            feat[col] = feat[col].str.replace("__", "_")
            feat[col] = feat[col].str.replace("__", "_")

        #print(feat.head(3))

        #print(feat.columns)
        feat['pivot_col']= feat[colnames[0]] + "_" + feat[colnames[1]]  + "_" + met + "_" +  str(startm) + "_" + str(endm)
        feat = pd.pivot(feat , index=['LOS_APP_ID'], columns= "pivot_col", 
                        values=  met + '_' + str(startm) + "_" + str(endm)+ "_sum",
                      ).reset_index()
        
        
        #print(feat.columns)
        #feat.drop(["pivot_col"], axis=1,inplace = True )
        #print(feat.head(2))
        if pos == 0 and met ==metrics[0]:
            feat_vf = feat
        else:
            feat_vf = feat_vf.merge(feat, on ="LOS_APP_ID",how='outer')


        print(feat_vf.shape, feat.shape)



(16051, 133) (16051, 133)
(21991, 261) (17435, 129)
(25356, 395) (17888, 135)
(27483, 519) (18004, 125)
(29150, 648) (18203, 130)


In [142]:
feat_vf.to_pickle("features_disbursed_loans_bank_acc.pkl")
feat_vf.head(3)

pivot_col,LOS_APP_ID,AutoLoanPersonal_COP_DisbAmt_0_6,AutoLoanPersonal_NAB_DisbAmt_0_6,AutoLoanPersonal_NBF_DisbAmt_0_6,AutoLoanPersonal_PRB_DisbAmt_0_6,AutoLoanPersonal_RRB_DisbAmt_0_6,BusinessLoanAgainstBankDeposits_NAB_DisbAmt_0_6,BusinessLoanGeneral_COP_DisbAmt_0_6,BusinessLoanGeneral_FRB_DisbAmt_0_6,BusinessLoanGeneral_NAB_DisbAmt_0_6,...,TractorLoan_NAB_DisbAmt_24_30,TractorLoan_NBF_DisbAmt_24_30,TractorLoan_PRB_DisbAmt_24_30,Two_WheelerLoan_COP_DisbAmt_24_30,Two_WheelerLoan_NAB_DisbAmt_24_30,Two_WheelerLoan_NBF_DisbAmt_24_30,Two_WheelerLoan_PRB_DisbAmt_24_30,Two_WheelerLoan_RRB_DisbAmt_24_30,UsedCarLoan_NBF_DisbAmt_24_30,UsedCarLoan_PRB_DisbAmt_24_30
0,0CV792020R1500937089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0CV792024R1500968341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0CV792070R1500935321,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:

# Cmpiling granular and customer features
granular_feat  =pd.read_pickle("features_disbursed_loans_bank_acc.pkl")
compiled_feat = pd.read_pickle("features_disbursed_loans.pkl")
net_feat = granular_feat.merge(compiled_feat, on ="LOS_APP_ID",how= "outer" )
net_feat.to_pickle("features_disbursed_loans_all.pkl")
net_feat.shape,net_feat.LOS_APP_ID.nunique(), granular_feat.shape, compiled_feat.shape

((29150, 653), 29150, (29150, 648), (29150, 6))

# Compiling All features

In [2]:
f1  = pd.read_pickle("features_c_loans_all_lessthan20.pkl")
f2  = pd.read_pickle("features_c_loans_all_20_40.pkl")
f3  = pd.read_pickle("features_c_loans_all_40_80.pkl")
f4  = pd.read_pickle("features_c_loans_all_80_plus.pkl")

f = f1.append([f2, f3, f4])
print(f.shape,f.LOS_APP_ID.nunique(), f1.shape, f2.shape, f3.shape, f4.shape)

d1  = pd.read_pickle("features_disbursed_loans_all.pkl")
feature_v1 = f.merge(d1, on ="LOS_APP_ID",how= "outer" )
feature_v1 = feature_v1.fillna(0)
feature_v1.to_pickle("feature_compilation.pkl")
feature_v1.shape, d1.shape, f.shape,feature_v1.LOS_APP_ID.nunique()


C:\Users\romil\AppData\Local\Temp\ipykernel_17248\250279206.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  f = f1.append([f2, f3, f4])


(31532, 2503) 31532 (20421, 2182) (6124, 1993) (3528, 1516) (1459, 1270)


((31646, 3155), (29150, 653), (31532, 2503), 31646)

In [3]:
feature_v1.head()

,LOS_APP_ID,AutoLoanPersonal_COP_DPD_0_6,AutoLoanPersonal_FRB_DPD_0_6,AutoLoanPersonal_MFI_DPD_0_6,AutoLoanPersonal_NAB_DPD_0_6,AutoLoanPersonal_NBF_DPD_0_6,AutoLoanPersonal_PRB_DPD_0_6,AutoLoanPersonal_RRB_DPD_0_6,BusinessLoanAgainstBankDeposits_COP_DPD_0_6,BusinessLoanAgainstBankDeposits_NAB_DPD_0_6,...,Two_WheelerLoan_NBF_DisbAmt_24_30,Two_WheelerLoan_PRB_DisbAmt_24_30,Two_WheelerLoan_RRB_DisbAmt_24_30,UsedCarLoan_NBF_DisbAmt_24_30,UsedCarLoan_PRB_DisbAmt_24_30,DisbAmt_0_6_sum,DisbAmt_6_12_sum,DisbAmt_12_18_sum,DisbAmt_18_24_sum,DisbAmt_24_30_sum
0,0CV792020R1500937089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0CV792024R1500936118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0CV792024R1500968341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0CV792024S1500935343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0CV792067O1500943522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
driverset= pd.read_pickle('driverset_18M.pkl')
driverset.head()

,LOS_APP_ID,DPD,DPD_1_plus_18,DPD_30_plus_18,DPD_60_plus_18
0,0CV792020R1500937089,0.0,0,0,0
1,0CV792024R1500936118,116.0,1,1,1
2,0CV792024R1500942080,0.0,0,0,0
3,0CV792024R1500968341,34.0,1,1,0
4,0CV792024S1500935343,0.0,0,0,0


In [8]:
base_data = driverset.merge(feature_v1, on = "LOS_APP_ID" , how = "inner")
base_data.to_pickle("modelbase_CV.pkl")

base_data.shape, base_data.LOS_APP_ID.nunique(), driverset.shape,feature_v1.shape



((31286, 3159), 31286, (34305, 5), (31646, 3155))

In [2]:
base_data=pd.read_pickle("modelbase_CV.pkl")
base_data.shape

(31286, 3159)

In [3]:
base_data.shape, base_data.DPD_1_plus_18.mean() , base_data.DPD_30_plus_18.mean(), base_data.DPD_60_plus_18.mean()

((31286, 3159), 0.538195998210062, 0.3024355941954868, 0.17237742121076519)